In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install scanpy
import scanpy as sc
import anndata

import scipy 


import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 70
plt.style.use('dark_background')

import seaborn as sns

from sklearn.decomposition import PCA

In [ ]:
    str_data_inf = ' GSE64016_human_H1hESC_Fucci_Y2014_460Cells_GEO_counts_Leng_Kendziorski '
    path_and_fn = '/kaggle/input/scrnaseq-hesc-h1-cell-line-fucci-cell-cycle-label/GSE64016_human_H1hESC_Fucci_Y2014_460Cells_GEO_counts_Leng_Kendziorski.h5ad'
    adata_orig = sc.read(path_and_fn)
    adata = adata_orig.copy()
    adata

In [ ]:
adata.obs.columns = ['Cell type', 'Cell cycle phase']# = adata.obs['Fucci cell cycle phase'] 
adata

In [ ]:
adata.obs[ 'Cell cycle phase'].value_counts()

In [ ]:
adata.uns['Cell cycle phase labels'] = True # 

adata.uns['Id'] = 'GSE64016'# 'GSE146773'  # Will be used for filename - can be GSE, GSM, 10xGenomics, etc 

#adata.uns['GSM'] =  ''
adata.uns['GSE'] = 'GSE64016' # adata.uns['Id'] #   'GSE146773' # adata.uns['Id']
adata.uns['Brief Info'] = 'H1 hESC Fucci and WT'# 'U2OS FUCCI' # Might be used for figures annotations
adata.uns['Link'] = r'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE64016'  #Link to data:  r'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE146773'  
adata.uns['Cellosaurus'] = r'https://web.expasy.org/cellosaurus/CVCL_9771'# r'https://web.expasy.org/cellosaurus/CVCL_0042'
#adata.uns['Doubling time string'] = r''# r'Cellosaurus: ~25-30 hours (DSMZ).'
#adata.uns['Doubling hours'] = np.nan# 27 #


adata.uns['Date'] =  '2014 12 09'# '2020 03 11' # Earliest date 

adata.uns['Cancer Cells'] =  0# 1 or 0  cancer cell or not #

adata.uns['Organism'] = 'Human' # 'Human' 
adata.uns['Cell type'] = 'H1hESC' # 'U2OS'
adata.uns['Cell further info'] = 'FUCCU' # Additional info - drug, 'FUCCI' or other special condtion
adata.uns['Year'] = 2014 # 2020 
adata.uns['Cell Count'] = adata.X.shape[0]
adata.uns['Source'] = 'GEO'# 'GEO' # where data were taken # 
adata.uns['Counts or preprocessing'] =  'Counts' # counts or preprocessed 
adata.uns['First author'] = 'Leng' # First author for paper if knownMahdessian
adata.uns['Last author'] = 'Kendziorski' # Last author for paper if known Lundberg 


adata.uns['Paper Link'] = r'https://pubmed.ncbi.nlm.nih.gov/26301841/'# r'https://www.nature.com/articles/s41586-021-03232-9'
adata.uns['Paper Title'] = 'Oscope identifies oscillatory genes in unsynchronized single-cell RNA-seq experiments'# 'Spatiotemporal dissection of the cell cycle with single-cell proteogenomics'
adata.uns['Abstract'] = '''Oscillatory gene expression is fundamental to development, but technologies for monitoring expression oscillations are limited. We have developed a statistical approach called Oscope to identify and characterize the transcriptional dynamics of oscillating genes in single-cell RNA-seq data from an unsynchronized cell population. Applying Oscope to a number of data sets, we demonstrated its utility and also identified a potential artifact in the Fluidigm C1 platform.''' #  '''The cell cycle, over which cells grow and divide, is a fundamental process of life. Its dysregulation has devastating consequences, including cancer1,2,3. The cell cycle is driven by precise regulation of proteins in time and space, which creates variability between individual proliferating cells. To our knowledge, no systematic investigations of such cell-to-cell proteomic variability exist. Here we present a comprehensive, spatiotemporal map of human proteomic heterogeneity by integrating proteomics at subcellular resolution with single-cell transcriptomics and precise temporal measurements of individual cells in the cell cycle. We show that around one-fifth of the human proteome displays cell-to-cell variability, identify hundreds of proteins with previously unknown associations with mitosis and the cell cycle, and provide evidence that several of these proteins have oncogenic functions. Our results show that cell cycle progression explains less than half of all cell-to-cell variability, and that most cycling proteins are regulated post-translationally, rather than by transcriptomic cycling. These proteins are disproportionately phosphorylated by kinases that regulate cell fate, whereas non-cycling proteins that vary between cells are more likely to be modified by kinases that regulate metabolism. This spatially resolved proteomic map of the cell cycle is integrated into the Human Protein Atlas and will serve as a resource for accelerating molecular studies of the human cell cycle and cell proliferation.'''
adata.uns['PMID'] = 26301841 # 

adata.uns['Comment'] = r'Reformating scripts to h5ad can be found at: https://www.kaggle.com/alexandervc/saving-adata-with-comments-and-long-fnames-u1-fuc' 
adata.uns['Link to our collection'] = r'https://www.kaggle.com/alexandervc/scrnaseq-datasets-collection' 

def get_filename(adata):
    fn = ''
    for t in [ 'Id', 'Organism', 'Cell type', 'Cell further info', 'Year', 'Cell Count', 'Source', 
              'Counts or preprocessing', 'First author', 'Last author']:
        if t == 'Year':
            fn += 'Y'+str(adata.uns[t])
        elif t == 'Cell Count':
            fn += str(adata.uns[t]) + 'Cells'
        else:
            fn += adata.uns[t]
        if t != 'Last author': fn += '_'

    fn += '.h5ad'
    return fn

fn = get_filename(adata)
fn

In [ ]:
adata.write_h5ad(fn,compression='gzip')
adata